In [ ]:
# prerequisite: 
# TODO

In [ ]:
# Here we ensure that the text sizes of our figures scale with the figures themselves.
# A change of figure width, which has to fit into column in latex document, affects perceived 
# text size; change of figure height does not affect perceived text size, provided the figure 
# height does not exceed the column height of the latex document

pt2inch = 72.27
w_onecol = 238.96417/pt2inch # numerator from \the\textwidth in relevant environment in latex document
w_twocol = 495.0/pt2inch     # numerator from \the\textwidth in relevant environment in latex document
wh_ratio = 6.4/4.8           # default width to height ratio in matplotlib

# height_scale can be changed with impunity. It allows for changing of the width to height 
#   ratio, without altering the perceived text sizes of the figures
# fig_scale can be changed with impunity. It simply changes the overall scale of the output 
#   figures, e.g. inside Jupyter Notebook and output .pdf files.
def prepare_one_column_figure(height_scale=1.0, fig_scale=1.0):
    plt.rc("pdf", fonttype=42)
    plt.rc("axes", labelsize=fig_scale*12)
    plt.rc("xtick", labelsize=fig_scale*12, top=True, direction="in")
    plt.rc("ytick", labelsize=fig_scale*12, right=True, direction="in")
    plt.rc("axes", titlesize=fig_scale*12)
    plt.rc("legend", fontsize=fig_scale*12, title_fontsize=fig_scale*12)
    plt.rc("font", size=fig_scale*12)
    return plt.figure(figsize=(fig_scale*w_onecol, fig_scale*height_scale*w_onecol/wh_ratio))

def prepare_two_column_figure(height_scale=1.0, fig_scale=1.0, nrows=1, ncols=1, 
                              sharex=False, sharey=False, gridspec_kw=None):
    plt.rc("pdf", fonttype=42)
    plt.rc("axes", labelsize=fig_scale*12)
    plt.rc("xtick", labelsize=fig_scale*12, top=True, direction="in")
    plt.rc("ytick", labelsize=fig_scale*12, right=True, direction="in")
    plt.rc("axes", titlesize=fig_scale*12)
    plt.rc("legend", fontsize=fig_scale*12, title_fontsize=fig_scale*12)
    plt.rc("font", size=fig_scale*12)
    if nrows == 1 and ncols == 1:
        return plt.figure(figsize=(fig_scale*w_twocol, fig_scale*height_scale*w_twocol/wh_ratio))
    else:
        return plt.subplots(nrows=nrows, ncols=ncols, 
                            figsize=(fig_scale*w_twocol, fig_scale*height_scale*w_twocol/wh_ratio), 
                            sharex=sharex, sharey=sharey, gridspec_kw=gridspec_kw)

In [ ]:
import uproot
import awkward as ak

import subprocess

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit
from scipy.stats import norm
from scipy.stats import uniform

import os
curdir = os.getcwd()
while not os.path.exists(".projectroot") and os.getcwd() != os.path.expanduser('~'):
    os.chdir("..")
try:
    proot = open(".projectroot").readline()
except:
    print("Could not find file '.projectroot'.")
#     print("Fallback: Look for the file 'rpad.py' in the current working directory (%s)." % curdir)
    print("Fonts in figures might not appear as intended, either.")
os.chdir(curdir)
import sys
import matplotlib.font_manager as font_manager
if len(proot) > 0:
#     sys.path.append(proot+"/python")
    font_dir = proot + '/fonts'
    font_files = font_manager.findSystemFonts(font_dir)
    font_files = font_manager.findSystemFonts()
    #for f in font_files:
    #    font_manager.FontManager.addfont(font_manager.fontManager, path=f)
    #    print(f)
    plt.rcParams.update({"font.family": "serif", "font.serif": "Linux Libertine O", "font.cursive": "Linux Libertine O", "font.sans-serif": "Linux Libertine O", "mathtext.fontset": "custom"})
# from rpad import rpad

os.makedirs("tmp", exist_ok=True)

dssds = ["U3", "U4", "U5"]
ids = [1, 2, 3]

# Load data

In [ ]:
al22files = []
seqs = [[30, 35], [40, 40], [45, 52], [55, 55], [57, 59], [61, 61], [63, 74]]
for i in range(len(seqs)):
    for j in range(seqs[i][0], seqs[i][1] + 1):
        al22files.append(proot + "/data/b2p/" + str(j).zfill(3) + "_*mlio.root:a")
p26files = []
seqs = [[79, 82], [84, 97], [103, 118]]
for i in range(len(seqs)):
    for j in range(seqs[i][0], seqs[i][1] + 1):
        p26files.append(proot + "/data/b2p/" + str(j).zfill(3) + "_*mlio.root:a")

In [ ]:
EAl = Q2pAl = ThetaAl = np.array([])
for batch in uproot.iterate(al22files, expressions=["E", "Q2p", "Theta"]):
    EAl     = np.append(EAl, ak.flatten(batch.E))
    Q2pAl   = np.append(Q2pAl, batch.Q2p)
    ThetaAl = np.append(ThetaAl, batch.Theta)
EAl = np.array(EAl[~np.isnan(EAl)])

EP = Q2pP = ThetaP = np.array([])
for batch in uproot.iterate(p26files, expressions=["E", "Q2p", "Theta"]):
    EP     = np.append(EP, ak.flatten(batch.E))
    Q2pP   = np.append(Q2pP, batch.Q2p)
    ThetaP = np.append(ThetaP, batch.Theta)
EP = np.array(EP[~np.isnan(EP)])

In [ ]:
E1Al = E2Al = np.array([])
for i in range(len(Q2pAl)):
    E1Al = np.append(E1Al, EAl[2*i])
    E2Al = np.append(E2Al, EAl[2*i+1])
E1P = E2P = np.array([])
for i in range(len(Q2pP)):
    E1P = np.append(E1P, EP[2*i])
    E2P = np.append(E2P, EP[2*i+1])

In [ ]:
print(len(EAl))
print(len(E1Al))
print(len(E2Al))
print(len(EP))
print(len(E1P))
print(len(E2P))

In [ ]:
E1Al  = E1Al/1e3
E2Al  = E2Al/1e3
E1P   = E1P/1e3
E2P   = E2P/1e3
Q2pAl = Q2pAl/1e3
Q2pP  = Q2pP/1e3

# Q2p vs. Ei

In [ ]:
%matplotlib widget

In [ ]:
fig, axes = prepare_two_column_figure(height_scale=0.7, fig_scale=1.5, nrows=1, ncols=2,
                                      gridspec_kw={'wspace': 0.01, 'width_ratios': [2, 1]})
plt.sca(axes[0])
plt.plot(E1Al, Q2pAl, 'k.', ms=0.8)
plt.plot(E2Al, Q2pAl, 'k.', ms=0.8)
plt.xlabel("$E_{i}$ (MeV)")
plt.ylabel("$Q_{2 \mathrm{p} }$ (MeV)")
low, high = plt.ylim()

plt.sca(axes[1])
plt.hist(Q2pAl, bins=np.arange(low, high+0.01, 0.01),
         color='k', lw=0.5, histtype='step', orientation='horizontal')
plt.xlabel('Counts / 10 keV')
plt.ylabel("")
axes[1].yaxis.tick_right()
axes[1].yaxis.set_ticks_position('both')
axes[1].yaxis.set_label_position("right")
plt.savefig('Q2p_E_22Al.pdf', bbox_inches='tight')

# fig, axes = plt.subplots(1, 2, figsize=(3*6.4/2, 4.8), width_ratios=[2, 1])
# plt.subplots_adjust(wspace=0.01)
# plt.sca(axes[0])
# plt.plot(E, Q2p, 'k.', ms=0.8)
# plt.xlabel("$E_{i}$ (MeV)")
# plt.ylabel("$Q_{2 \mathrm{p} }$ (MeV)")
# plt.xticks(np.arange(0, 11, 1))
# plt.xticks(np.arange(0, 11, 0.5), minor=True)
# plt.yticks(np.arange(0, 11, 1))
# plt.yticks(np.arange(0, 11, 0.5), minor=True)
# plt.xlim(0, Eupp)
# plt.ylim(0, Qupp)
# plt.ylim(Qlow, Qupp)
# plt.sca(axes[1])
# plt.hist(q2p, bins=np.linspace(Qlow, Qupp, bins), color='k', lw=0.5, ls='-', histtype='step', 
#          orientation='horizontal')
# plt.xlabel('Counts / %.1f keV' % dQ)
# plt.xticks(np.arange(cstep, cupp, cstep))
# plt.ylabel("")
# axes[1].yaxis.tick_right()
# axes[1].yaxis.set_ticks_position('both')
# axes[1].yaxis.set_label_position("right")
# plt.yticks(np.arange(0, 11, 1))
# plt.yticks(np.arange(0, 11, 0.5), minor=True)
# plt.ylim(Qlow, Qupp)
# #plt.text(0.8, 0.8, ann, fontsize=18, ha='center', va='center', transform=plt.gca().transAxes)
# plt.savefig('%s.png' % name, dpi=300, bbox_inches='tight')
# #plt.text(0.5, 0.3, ann, fontsize=18, ha='center', va='center', transform=plt.gca().transAxes)
# #plt.savefig('%s_Q2p_E.png' % name, dpi=300, bbox_inches='tight')
# plt.show()

In [ ]:
fig, axes = prepare_two_column_figure(height_scale=0.7, fig_scale=1.5, nrows=1, ncols=2,
                                      gridspec_kw={'wspace': 0.01, 'width_ratios': [2, 1]})
plt.sca(axes[0])
plt.plot(E1P[Q2pP < 10], Q2pP[Q2pP < 10], 'k.', ms=0.8)
plt.plot(E2P[Q2pP < 10], Q2pP[Q2pP < 10], 'k.', ms=0.8)
plt.xlabel("$E_{i}$ (MeV)")
plt.ylabel("$Q_{2 \mathrm{p} }$ (MeV)")
low, high = plt.ylim()

plt.sca(axes[1])
plt.hist(Q2pP[Q2pP < 10], bins=np.arange(low, high+0.01, 0.01),
         color='k', lw=0.5, histtype='step', orientation='horizontal')
plt.xlabel('Counts / 10 keV')
plt.ylabel("")
axes[1].yaxis.tick_right()
axes[1].yaxis.set_ticks_position('both')
axes[1].yaxis.set_label_position("right")
plt.savefig('Q2p_E_26P.pdf', bbox_inches='tight')

# Old

In [ ]:
import uproot
import awkward as ak

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

scale=1.25

plt.rc("pdf", fonttype=42)
plt.rc("pdf", fonttype=42)
plt.rc("axes", labelsize=0.9*scale*20)
plt.rc("xtick", labelsize=scale*16, top=True, direction="in")
plt.rc("ytick", labelsize=scale*16, right=True, direction="in")
plt.rc("axes", titlesize=scale*20)
plt.rc("legend", fontsize=14)

import os
curdir = os.getcwd()
while not os.path.exists(".projectroot") and os.getcwd() != os.path.expanduser('~'):
    os.chdir("..")
try:
    proot = open(".projectroot").readline()
except:
    print("Could not find file '.projectroot'.")
    print("Fallback: Look for the file 'rpad.py' in the current working directory (%s)." % curdir)
    print("Fonts in figures might not appear as intended, either.")
os.chdir(curdir)
import sys
import matplotlib.font_manager as font_manager
if len(proot) > 0:
    sys.path.append(proot+"/python")
    font_dir = proot + '/fonts'
    font_files = font_manager.findSystemFonts(font_dir)
    for f in font_files:
        font_manager.FontManager.addfont(font_manager.fontManager, path=f)
    plt.rcParams.update({"font.family": "serif", "font.serif": "Linux Libertine O", "font.cursive": "Linux Libertine O", "font.sans-serif": "Linux Libertine O", "mathtext.fontset": "custom"})
from rpad import rpad

os.makedirs("tmp", exist_ok=True)

In [ ]:
for name, Qupp, Qlow, bins, ann, Eupp, cupp, cstep in zip(("22al_28h", "26p_24h"), (8.0, 10.3), (0.6, 0.6), 
                                                   (500, 810), ("$\mathrm{^{22}Al}$", "$\mathrm{^{26}P}$"), 
                                                   (6.5, 8.5), (70, 201), (20, 50)):
    f = uproot.open(proot+"/data/b2p/%s.root" % name)
    t = f["a"]

    q2p = t.arrays(["Q2p"], "E > 50").Q2p[:,0]/1e3
    a = t.arrays(["Q2p", "E"], "E > 50")
    Q2p = ak.flatten(a.Q2p/1e3)
    E   = ak.flatten(a.E/1e3)
    dQ = 1e3*(Qupp - Qlow)/bins

    fig, axes = plt.subplots(1, 2, figsize=(3*6.4/2, 4.8), width_ratios=[2, 1])
    plt.subplots_adjust(wspace=0.01)
    plt.sca(axes[0])
    plt.plot(E, Q2p, 'k.', ms=0.8)
    plt.xlabel("$E_{i}$ (MeV)")
    plt.ylabel("$Q_{2 \mathrm{p} }$ (MeV)")
    plt.xticks(np.arange(0, 11, 1))
    plt.xticks(np.arange(0, 11, 0.5), minor=True)
    plt.yticks(np.arange(0, 11, 1))
    plt.yticks(np.arange(0, 11, 0.5), minor=True)
    plt.xlim(0, Eupp)
    plt.ylim(0, Qupp)
    plt.ylim(Qlow, Qupp)
    plt.sca(axes[1])
    plt.hist(q2p, bins=np.linspace(Qlow, Qupp, bins), color='k', lw=0.5, ls='-', histtype='step', 
             orientation='horizontal')
    plt.xlabel('Counts / %.1f keV' % dQ)
    plt.xticks(np.arange(cstep, cupp, cstep))
    plt.ylabel("")
    axes[1].yaxis.tick_right()
    axes[1].yaxis.set_ticks_position('both')
    axes[1].yaxis.set_label_position("right")
    plt.yticks(np.arange(0, 11, 1))
    plt.yticks(np.arange(0, 11, 0.5), minor=True)
    plt.ylim(Qlow, Qupp)
    #plt.text(0.8, 0.8, ann, fontsize=18, ha='center', va='center', transform=plt.gca().transAxes)
    plt.savefig('%s.png' % name, dpi=300, bbox_inches='tight')
    #plt.text(0.5, 0.3, ann, fontsize=18, ha='center', va='center', transform=plt.gca().transAxes)
    #plt.savefig('%s_Q2p_E.png' % name, dpi=300, bbox_inches='tight')
    plt.show()
    
    f.close()

In [ ]:
oa = np.loadtxt(proot + "/data/oa_vs_sa/theta.dat")
sa = np.loadtxt(proot + "/data/oa_vs_sa/omega.dat")

In [ ]:
f = uproot.open(proot+"/data/tmp/q2p/%s/%s.root" % ("22al_28h", "22al_28h"))
t = f["a"]
vs = np.rad2deg(t.arrays(["Theta"], "E > 50").Theta[:,0])

weights = np.array([])
for v in vs:
    absdiff = 424242
    closest = None
    for i in range(len(oa)):
        if abs(v - oa[i]) < absdiff:
            absdiff = abs(v - oa[i])
            closest = i
    #print(closest)
    weights = np.append(weights, 1/sa[closest])

dv = 5
plt.hist(vs, bins=np.arange(0, 180+1, dv), weights=weights, color='k', lw=0.8, ls='-', histtype='step')
plt.xlabel('$\Theta_{\mathrm{2p}}$ (deg.)')
plt.ylabel('$\Omega$ (arb.)')
plt.savefig("22al_V2p.png", dpi=300, bbox_inches='tight')
plt.show()
plt.hist(np.cos(vs), bins=np.linspace(-1+0.4, +1-0.4, int(180/dv)), weights=weights, color='k', lw=0.8, ls='-', histtype='step')
plt.xlabel('$\cos{\Theta_{\mathrm{2p}}}$ (deg.)')
plt.ylabel('$\Omega$ (arb.)')
plt.savefig("22al_cosV2p.png", dpi=300, bbox_inches='tight')
plt.show()





f = uproot.open(proot+"/data/tmp/q2p/%s/%s.root" % ("26p_24h", "26p_24h"))
t = f["a"]
vs = np.rad2deg(t.arrays(["Theta"], "E > 50").Theta[:,0])


weights = np.array([])
for v in vs:
    absdiff = 424242
    closest = None
    for i in range(len(oa)):
        if abs(v - oa[i]) < absdiff:
            absdiff = abs(v - oa[i])
            closest = i
    #print(closest)
    weights = np.append(weights, 1/sa[closest])

dv = 5
plt.hist(vs, bins=np.arange(0, 180+1, dv), weights=weights, color='k', lw=0.8, ls='-', histtype='step')
plt.xlabel('$\Theta_{\mathrm{2p}}$ (deg.)')
plt.ylabel('$\Omega$ (arb.)')
plt.savefig("26p_V2p.png", dpi=300, bbox_inches='tight')
plt.show()
plt.hist(np.cos(vs), bins=np.linspace(-1+0.4, +1-0.4, int(180/dv)), weights=weights, color='k', lw=0.8, ls='-', histtype='step')
plt.xlabel('$\cos{\Theta_{\mathrm{2p}}}$ (deg.)')
plt.ylabel('$\Omega$ (arb.)')
plt.savefig("26p_cosV2p.png", dpi=300, bbox_inches='tight')
plt.show()